In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords

## Intro to word2vec

The most common unsupervised neural network approach for NLP is word2vec, a shallow neural network model for converting words to vectors using distributed representation: Each word is represented by many neurons, and each neuron is involved in representing many words.  At the highest level of abstraction, word2vec assigns a vector of random values to each word.  For a word W, it looks at the words that are near W in the sentence, and shifts the values in the word vectors such that the vectors for words near that W are closer to the W vector, and vectors for words not near W are farther away from the W vector.  With a large enough corpus, this will eventually result in words that often appear together having vectors that are near one another, and words that rarely or never appear together having vectors that are far away from each other.  Then, using the vectors, similarity scores can be computed for each pair of words by taking the cosine of the vectors.  

This may sound quite similar to the Latent Semantic Analysis approach you just learned.  The conceptual difference is that LSA creates vector representations of sentences based on the words in them, while word2vec creates representations of individual words, based on the words around them.

## What is it good for?

Word2vec is useful for any time when computers need to parse requests written by humans. The problem with human communication is that there are so many different ways to communicate the same concept. It's easy for us, as humans, to know that "the silverware" and "the utensils" can refer to the same thing. Computers can't do that unless we teach them, and this can be a real chokepoint for human/computer interactions. If you've ever played a text adventure game (think _Colossal Cave Adventure_ or _Zork_), you may have encountered the following scenario:

And your brain explodes from frustration. A text adventure game that incorporates a properly trained word2vec model would have vectors for "pick up", "lift", and "take" that are close to the vector for "grab" and therefore could accept those other verbs as synonyms so you could move ahead faster. In more practical applications, word2vec and other similar algorithms are what help a search engine return the best results for your query and not just the ones that contain the exact words you used. In fact, search is a better example, because not only does the search engine need to understand your request, it also needs to match it to web pages that were _also written by humans_ and therefore _also use idiosyncratic language_.

Humans, man.  

So how does it work?

## Generating vectors: Multiple algorithms

In considering the relationship between a word and its surrounding words, word2vec has two options that are the inverse of one another:

 * _Continuous Bag of Words_ (CBOW): the identity of a word is predicted using the words near it in a sentence.
 * _Skip-gram_: The identities of words are predicted from the word they surround. Skip-gram seems to work better for larger corpuses.

For the sentence "Terry Gilliam is a better comedian than a director", if we focus on the word "comedian" then CBOW will try to predict "comedian" using "is", "a", "better", "than", "a", and "director".  Skip-gram will try to predict "is", "a", "better", "than", "a", and "director" using the word "comedian". In practice, for CBOW the vector for "comedian" will be pulled closer to the other words, while for skip-gram the vectors for the other words will be pulled closer to "comedian".  

In addition to moving the vectors for nearby words closer together, each time a word is processed some vectors are moved farther away. Word2vec has two approaches to "pushing" vectors apart:
 
 * _Negative sampling_: Like it says on the tin, each time a word is pulled toward some neighbors, the vectors for a randomly chosen small set of other words are pushed away.
 * _Hierarchical softmax_: Every neighboring word is pulled closer or farther from a subset of words chosen based on a tree of probabilities.

## What is similarity? Word2vec strengths and weaknesses

Keep in mind that word2vec operates on the assumption that frequent proximity indicates similarity, but words can be "similar" in various ways. They may be conceptually similar ("royal", "king", and "throne"), but they may also be functionally similar ("tremendous" and "negligible" are both common modifiers of "size"). Here is a more detailed exploration, [with examples](https://quomodocumque.wordpress.com/2016/01/15/messing-around-with-word2vec/), of what "similarity" means in word2vec.

One cool thing about word2vec is that it can identify similarities between words _that never occur near one another in the corpus_. For example, consider these sentences:

"The dog played with an elastic ball."
"Babies prefer the ball that is bouncy."
"I wanted to find a ball that's elastic."
"Tracy threw a bouncy ball."

"Elastic" and "bouncy" are similar in meaning in the text but don't appear in the same sentence. However, both appear near "ball". In the process of nudging the vectors around so that "elastic" and "bouncy" are both near the vector for "ball", the words also become nearer to one another and their similarity can be detected.

For a while after it was introduced, [no one was really sure why word2vec worked as well as it did](https://arxiv.org/pdf/1402.3722v1.pdf) (see last paragraph of the linked paper). A few years later, some additional math was developed to explain word2vec and similar models. If you are comfortable with both math and "academese", have a lot of time on your hands, and want to take a deep dive into the inner workings of word2vec, [check out this paper](https://arxiv.org/pdf/1502.03520v7.pdf) from 2016.  

One of the draws of word2vec when it first came out was that the vectors could be used to convert analogies ("king" is to "queen" as "man" is to "woman", for example) into mathematical expressions ("king" + "woman" - "man" = ?) and solve for the missing element ("queen"). This is kinda nifty.

A drawback of word2vec is that it works best with a corpus that is at least several billion words long. Even though the word2vec algorithm is speedy, this is a a lot of data and takes a long time! Our example dataset is only two million words long, which allows us to run it in the notebook without overwhelming the kernel, but probably won't give great results.  Still, let's try it!

There are a few word2vec implementations in Python, but the general consensus is the easiest one to us is in [gensim](https://radimrehurek.com/gensim/models/word2vec.html). Now is a good time to `pip install gensim` if you don't have it yet.

In [3]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    return text[0:900000]


# Import all the Austen in the Project Gutenberg corpus.
austen = ""
for novel in ['persuasion','emma','sense']:
    work = gutenberg.raw('austen-' + novel + '.txt')
    austen = austen + work

# Clean the data.
austen_clean = text_cleaner(austen)

In [4]:
# Parse the data. This can take some time.
nlp = spacy.load('en')
austen_doc = nlp(austen_clean)

In [5]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences = []
for sentence in austen_doc.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences.append(sentence)


print(sentences[20])
print('We have {} sentences and {} tokens.'.format(len(sentences), len(austen_clean)))

['daughter', 'eld', 'give', 'thing', 'tempt']
We have 8146 sentences and 900000 tokens.


In [6]:
import gensim
from gensim.models import word2vec

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [7]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

[('goddard', 0.9406564831733704), ('benwick', 0.9364099502563477), ('harville', 0.9233611822128296), ('musgrove', 0.9052064418792725), ('weston', 0.8826019167900085), ('wentworth', 0.8774406313896179), ('clay', 0.8688079118728638), ('colonel', 0.8620197176933289), ('drawing', 0.8536796569824219), ('yes', 0.8382086157798767)]


In [8]:
# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('mr', 'mrs'))

0.94267935


In [9]:
# One of these things is not like the other...
print(model.wv.doesnt_match("breakfast marriage dinner lunch".split()))

dinner


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


Clearly this model is not great – while some words given above might possibly fill in the analogy woman:lady::man:?, most answers likely make little sense. You'll notice as well that re-running the model likely gives you different results, indicating random chance plays a large role here.

We do, however, get a nice result on "marriage" being dissimilar to "breakfast", "lunch", and "dinner". 

## Drill 0

Take a few minutes to modify the hyperparameters of this model and see how its answers change. Can you wrangle any improvements?

In [10]:
import gensim
from gensim.models import word2vec

In [11]:
# Tinker with hyperparameters here.
def model_run(sentences,**kwargs):
    workers = kwargs.get('workers',4) # Number of threads to run in parallel (if your computer does parallel processing).
    min_count = kwargs.get('min_count',10) # Minimum word count threshold.
    window = kwargs.get('window',6)        # Number of words around target word to consider.
    sg = kwargs.get('sg',0)                # Use CBOW (0) because our corpus is small.
    sample = kwargs.get('sample',1e-3)     # Penalize frequent words.
    size = kwargs.get('size',300)          # Word vector length.
    hs = kwargs.get('hs',1)                # Use hierarchical softmax.

    model = word2vec.Word2Vec(
        sentences,
        workers=workers,     
        min_count=min_count,   
        window=window,      
        sg=sg,          
        sample=sample ,  
        size=size,      
        hs=hs          
    )
    print(model)
    print('done!')

    # List of words in model.
    vocab = model.wv.vocab.keys()

    print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))
    print('\n')
    # Similarity is calculated using the cosine, so again 1 is total
    # similarity and 0 is no similarity.
    print("Similarity of 'mr' and 'mrs': ", model.wv.similarity('mr', 'mrs'))
    print('\n')
    # One of these things is not like the other...
    oottinlto = "breakfast marriage dinner lunch"
    print("Which of these words doesn't match the others? ",oottinlto,'\n',model.wv.doesnt_match(oottinlto.split()))

In [12]:
# Using default values
model_run(sentences)

Word2Vec(vocab=1138, size=300, alpha=0.025)
done!
[('clay', 0.9570316672325134), ('musgrove', 0.9454711675643921), ('goddard', 0.9366889595985413), ('benwick', 0.9308604001998901), ('harville', 0.9185048341751099), ('wentworth', 0.8978030681610107), ('hall', 0.8325870037078857), ('colonel', 0.8260571956634521), ('smith', 0.8184881806373596), ('weston', 0.8061772584915161)]


Similarity of 'mr' and 'mrs':  0.96014816


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 breakfast


In [13]:
# Switch from CBOW to skipgram.
model_run(sentences,sg=1)

Word2Vec(vocab=1138, size=300, alpha=0.025)
done!
[('engage', 0.6899467706680298), ('except', 0.6637136936187744), ('hall', 0.6422021389007568), ('mention', 0.6377698183059692), ('prefer', 0.6299887895584106), ('honour', 0.6288518905639648), ('remove', 0.6276795268058777), ('tired', 0.627098798751831), ('carriage', 0.6265672445297241), ('mary', 0.6259537935256958)]


Similarity of 'mr' and 'mrs':  0.6106744


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 marriage


In [14]:
# Increase/decrease the min_count
model_run(sentences,min_count=20)
model_run(sentences,min_count=5)

Word2Vec(vocab=649, size=300, alpha=0.025)
done!
[('benwick', 0.9397519826889038), ('colonel', 0.9379609227180481), ('goddard', 0.9171911478042603), ('cole', 0.9040539860725403), ('musgrove', 0.9021955132484436), ('door', 0.8991220593452454), ('probably', 0.8966006636619568), ('smith', 0.8953303694725037), ('dixon', 0.8947281837463379), ('harville', 0.8943551778793335)]


Similarity of 'mr' and 'mrs':  0.9369531


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 dinner
Word2Vec(vocab=1842, size=300, alpha=0.025)
done!
[('suddenly', 0.8862031698226929), ('hitherto', 0.88575279712677), ('harville', 0.8701974153518677), ('blame', 0.865852952003479), ('suitable', 0.8578872680664062), ('unaffected', 0.8508696556091309), ('apparently', 0.8433620929718018), ('wild', 0.842427134513855), ('hall', 0.8356485366821289), ('motion', 0.8341408371925354)]


Similarity of 'mr' and 'mrs':  0.8311087


Which of these words doesn't match the others?  breakfast marriage din

In [15]:
# Increase/decrease the window...?
model_run(sentences,window=10)
model_run(sentences,window=2)

Word2Vec(vocab=1138, size=300, alpha=0.025)
done!
[('benwick', 0.9229929447174072), ('goddard', 0.9199917316436768), ('harville', 0.893903374671936), ('wentworth', 0.8643467426300049), ('colonel', 0.8600151538848877), ('musgrove', 0.8412532210350037), ('weston', 0.8140028715133667), ('drawing', 0.8069411516189575), ('christmas', 0.8028655052185059), ('farm', 0.8009656667709351)]


Similarity of 'mr' and 'mrs':  0.9592606


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 marriage
Word2Vec(vocab=1138, size=300, alpha=0.025)
done!
[('goddard', 0.9113954901695251), ('hall', 0.875407874584198), ('clay', 0.8687660694122314), ('colonel', 0.8290461301803589), ('musgrove', 0.7916613817214966), ('weston', 0.7783045768737793), ('harville', 0.7713574171066284), ('cole', 0.7649472951889038), ('croft', 0.7549240589141846), ('yes', 0.7441867589950562)]


Similarity of 'mr' and 'mrs':  0.8016316


Which of these words doesn't match the others?  breakfast marriage dinn

Increasing the window improved the mr/mrs score and didn't seem to hurt anything else (but don't call me 'shirley'). Let's note that and see what changing other parameters can do.

In [16]:
# Increase/decrease the sample/penalization of frequent words.
model_run(sentences,sample=1e-2)
model_run(sentences,sample=1e-4)

Word2Vec(vocab=1138, size=300, alpha=0.025)
done!
[('clay', 0.7411788702011108), ('croft', 0.7165433764457703), ('musgrove', 0.6664475202560425), ('harville', 0.6506083607673645), ('benwick', 0.5952684879302979), ('hall', 0.5897868275642395), ('goddard', 0.5429544448852539), ('colonel', 0.49776867032051086), ('smith', 0.4825044870376587), ('wentworth', 0.47152048349380493)]


Similarity of 'mr' and 'mrs':  0.5655026


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 breakfast
Word2Vec(vocab=1138, size=300, alpha=0.025)
done!
[('little', 0.9998167753219604), ('go', 0.999812662601471), ('mrs', 0.9998104572296143), ('anne', 0.9998098611831665), ('great', 0.9998079538345337), ('leave', 0.9998047351837158), ('good', 0.9998037219047546), ('feel', 0.9998018741607666), ('see', 0.999801516532898), ('return', 0.9998013377189636)]


Similarity of 'mr' and 'mrs':  0.999882


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 breakfast

In [17]:
# Increase/decrease size/word vector length.
model_run(sentences,size=500)
model_run(sentences,size=100)

Word2Vec(vocab=1138, size=500, alpha=0.025)
done!
[('benwick', 0.9506795406341553), ('goddard', 0.9497581720352173), ('harville', 0.9379949569702148), ('musgrove', 0.9138126373291016), ('weston', 0.9129750728607178), ('drawing', 0.9025213718414307), ('clay', 0.8941812515258789), ('wentworth', 0.8852489590644836), ('colonel', 0.8846334218978882), ('smith', 0.8797270059585571)]


Similarity of 'mr' and 'mrs':  0.96318054


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 marriage
Word2Vec(vocab=1138, size=100, alpha=0.025)
done!
[('goddard', 0.8840848207473755), ('benwick', 0.8664734959602356), ('clay', 0.8538286685943604), ('harville', 0.8430572748184204), ('hall', 0.8297086954116821), ('winthrop', 0.8289786577224731), ('musgrove', 0.826827883720398), ('drawing', 0.8150631785392761), ('colonel', 0.8102028369903564), ('yes', 0.7904012203216553)]


Similarity of 'mr' and 'mrs':  0.89990544


Which of these words doesn't match the others?  breakfast marriag

In [18]:
model_run(sentences,size=3)

Word2Vec(vocab=1138, size=3, alpha=0.025)
done!
[('goddard', 0.9801400899887085), ('weston', 0.9668737649917603), ('cole', 0.9042726159095764), ('harville', 0.8944903016090393), ('wentworth', 0.8767645955085754), ('captain', 0.8598239421844482), ('yes', 0.8483148813247681), ('colonel', 0.8362590670585632), ('benwick', 0.8277834057807922), ('clay', 0.8204617500305176)]


Similarity of 'mr' and 'mrs':  0.95047176


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 breakfast


In [19]:
# Switch from hierarchical softmax to negative sampling
model_run(sentences,hs=0)

Word2Vec(vocab=1138, size=300, alpha=0.025)
done!
[('live', 0.9996018409729004), ('anne', 0.9995896220207214), ('self', 0.9995884299278259), ('character', 0.9995872974395752), ('perfectly', 0.999586820602417), ('view', 0.9995816946029663), ('new', 0.9995814561843872), ('eye', 0.9995810985565186), ('mean', 0.9995791912078857), ('pain', 0.9995783567428589)]


Similarity of 'mr' and 'mrs':  0.99971116


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 breakfast


That didn't feel like a very fruitful exercise. :(

# Example word2vec applications

You can use the vectors from word2vec as features in other models, or try to gain insight from the vector compositions themselves.

Here are some neat things people have done with word2vec:

 * [Visualizing word embeddings in Jane Austen's Pride and Prejudice](http://blogger.ghostweather.com/2014/11/visualizing-word-embeddings-in-pride.html). Skip to the bottom to see a _truly honest_ account of this data scientist's process.

 * [Tracking changes in Dutch Newspapers' associations with words like 'propaganda' and 'alien' from 1950 to 1990](https://www.slideshare.net/MelvinWevers/concepts-through-time-tracing-concepts-in-dutch-newspaper-discourse-using-sequential-word-vector-spaces).

 * [Helping customers find clothing items similar to a given item but differing on one or more characteristics](http://multithreaded.stitchfix.com/blog/2015/03/11/word-is-worth-a-thousand-vectors/).

## Drill 1: Word2Vec on 100B+ words

As we mentioned, word2vec really works best on a big corpus, but it can take half a day to clean such a corpus and run word2vec on it.  Fortunately, there are word2vec models available that have already been trained on _really_ big corpora. They are big files, but you can download a [pretrained model of your choice here](https://github.com/3Top/word2vec-api). At minimum, the ones built with word2vec (check the "Architecture" column) should load smoothly using an appropriately modified version of the code below, and you can play to your heart's content.

Because the models are so large, however, you may run into memory problems or crash the kernel. If you can't get a pretrained model to run locally, check out this [interactive web app of the Google News model](https://rare-technologies.com/word2vec-tutorial/#bonus_app) instead.

However you access it, play around with a pretrained model. Is there anything interesting you're able to pull out about analogies, similar words, or words that don't match? Write up a quick note about your tinkering and discuss it with your mentor during your next session.

In [20]:
# Load Google's pre-trained Word2Vec model.
# NOTE: I renamed this because it was called 'model', just like the model above. 
#       That caused some really confusing issues until I realized this!
model_g = gensim.models.KeyedVectors.load_word2vec_format ('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [22]:
# Play around with your pretrained model here.
print(model_g.most_similar(positive=['lady', 'man'], negative=['woman']))
print('\n')
# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print("Similarity of 'mr' and 'mrs': ", model_g.similarity('mr', 'mrs'))
print('\n')
# One of these things is not like the other...
oottinlto = "breakfast marriage dinner lunch"
print("Which of these words doesn't match the others? ",oottinlto,'\n',model_g.doesnt_match(oottinlto.split()))

[('fella', 0.6031545400619507), ('gentleman', 0.5849651098251343), ('chap', 0.5543248653411865), ('gent', 0.543907880783081), ('guy', 0.5265034437179565), ('lad', 0.5139425992965698), ('feller', 0.5072450041770935), ('bloke', 0.49030163884162903), ('rascal', 0.48736995458602905), ('ladies', 0.47617608308792114)]


Similarity of 'mr' and 'mrs':  0.66098833


Which of these words doesn't match the others?  breakfast marriage dinner lunch 
 marriage


In [35]:
print(model_g.most_similar(positive=['smile','wink'], negative=['mouth']))

[('grin', 0.6251746416091919), ('wry_smile', 0.5788321495056152), ('smirk', 0.5728423595428467), ('sly_grin', 0.5581395626068115), ('grins', 0.5403335094451904), ('smiles', 0.5341616868972778), ('chuckle', 0.5229692459106445), ('smiling', 0.5213208794593811), ('mischievous_smile', 0.5204308032989502), ('twinkle', 0.5198656916618347)]


Hey honestly I'm not good at analogies...

In [66]:
# One of these things is not like the other...
oottinlto = "speakers turntable sofa amplifier subwoofer"
print("Which of these words doesn't match the others? ",oottinlto,'\n',model_g.doesnt_match(oottinlto.split()))

Which of these words doesn't match the others?  speakers turntable sofa amplifier subwoofer 
 sofa


I tried a few other words like "screen" and "ethernet" in place of "sofa", but sometimes the model would kick out words I didn't expect, usually "speakers", as being less similar than the others.

I also see the similar_by_word function in an online tutorial. It's fun:

In [62]:
# A little Midwest flavor
model_g.similar_by_word('Younkers')

[('Carson_Pirie_Scott', 0.6203279495239258),
 ('Shopko', 0.61898273229599),
 ('Elder_Beerman', 0.6098634004592896),
 ('Famous_Barr', 0.6059889197349548),
 ('Southridge_Mall', 0.6040830612182617),
 ('Filene', 0.5973539352416992),
 ('ShopKo', 0.5928394794464111),
 ('Econofoods', 0.5855381488800049),
 ('Herbergers', 0.578551173210144),
 ('JC_Penney', 0.5745757818222046)]

These two models are obviously going to return very different results for "emma"--the one trained on Austen's works finds words related to Emma, the character. The Google model seems to mostly find other women's names.

In [75]:
model.wv.similar_by_word('emma')

[('smile', 0.9967248439788818),
 ('establish', 0.9953662753105164),
 ('shake', 0.9948168396949768),
 ('beg', 0.994679868221283),
 ('blush', 0.9939569234848022),
 ('ought', 0.9939529895782471),
 ('sake', 0.9926249384880066),
 ('caution', 0.9921291470527649),
 ('keep', 0.9917089939117432),
 ('wit', 0.9914932250976562)]

In [76]:
model_g.similar_by_word('emma')

[('joan', 0.6712832450866699),
 ('natalie', 0.6656539440155029),
 ('nigel', 0.6560487151145935),
 ('denise', 0.6524022221565247),
 ('jensen', 0.6521915793418884),
 ('gibbs', 0.6516698598861694),
 ('gemma', 0.6513988375663757),
 ('joel', 0.6508280634880066),
 ('caroline', 0.6500694751739502),
 ('becky', 0.6466513872146606)]

In general, the Google model, which has been trained with a much larger corpus, yields better results than one that has been trained on just a few works.